In [1]:
import requests
import json
from os import environ


Retrieving Project - testing API key

In [ ]:
project_params = "projects/Traffic Sign Detection"
url = "https://api.scale.com/v1/"

headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", url+project_params, headers=headers)
pretty_project_name = json.loads(response.text)
print (json.dumps(pretty_project_name, indent=4))


Retrieving Tasks

In [ ]:
task_params = "tasks?status=completed&project=Traffic Sign Detection"
response = requests.request("GET", url+task_params, headers=headers)

task_list_json = json.loads(response.text)

for i in task_list_json["docs"]:
    # print (json.dumps(i["task_id"], indent=4))
    response_task = requests.request("GET", url+"task/"+i["task_id"], headers=headers)
    response_task_json = json.loads(response_task.text)
    print (json.dumps(response_task_json, indent=4))

# Phase 2: one QC

QC 1: Check that visible signs do not have "NA" as background color

In [8]:

def check_visible_sign_bg_color(i):
    annotations=i['response']['annotations']
    # annotations is a list of different labels. 
    for j in annotations:
        # Check if background color of annotation is not_applicable
        if j['attributes']['background_color'] == 'not_applicable':
            # Check if annotation label is not "non visible face"
            if j['label'] !=  "non_visible_face":
                print('ERROR: Task id: {} annotation: {} type: {} cannot have background color as not_applicable.\n'.format(i["task_id"],j['uuid'],j['label']))
   

In [9]:
response = requests.request("GET", url+task_params, headers=headers)

task_list_json = json.loads(response.text)

for i in task_list_json["docs"]:
    # print (json.dumps(i["task_id"], indent=4))
    response_task = requests.request("GET", url+"task/"+i["task_id"], headers=headers)
    response_task_json = json.loads(response_task.text)
    # check_box_size(i)
    check_visible_sign_bg_color(i)


ERROR: Task id: 5f127f6c3a6b1000172320ad annotation: 8f9e0df3-7534-46ab-b7b4-dbae311c88cd type: traffic_control_sign cannot have background color as not_applicable.

ERROR: Task id: 5f127f6c3a6b1000172320ad annotation: dda85426-ba5f-4910-b92b-e50fc0bfb793 type: traffic_control_sign cannot have background color as not_applicable.

ERROR: Task id: 5f127f6c3a6b1000172320ad annotation: 0bc2ab2c-16b0-461e-8866-08f3bdc87238 type: traffic_control_sign cannot have background color as not_applicable.

ERROR: Task id: 5f127f6c3a6b1000172320ad annotation: fd7a83ed-cbcc-4b98-8746-98c7551cb4a2 type: policy_sign cannot have background color as not_applicable.



QC 2: Big boxes (which take up 20% of the image), should not be empty. Check if boxes bigger than 20% of the image do contain background color.

In [6]:
# verify if the bigger annotations have sufficient info within them.
def check_box_size(i):
    annotations=i['response']['annotations']
    # annotations is a list of different labels. 
    for j in annotations:
        # Check if bounding box is more than 20% of image size
        if j['width']*j['height'] > 0.2*1396*850:
            # Check if background color of annotation is not_applicable
            if j['attributes']['background_color'] == 'not_applicable':
                print('ERROR: Big box containing no information in {} tag of task {} '.format(j['uuid'],i['task_id'] ))
        